# Input Data Structure for RNNs

In [ ]:
import numpy as np
import sys
sys.path.append("..")
from moisture_rnn import staircase, staircase_2
import pandas as pd


## Background

RNNs are a type of timeseries model that relates the outcome at time $t$ to the outcome at previous times. Like other machine learning models, training is typically done by calculating the gradient of the output with respect to the weights, or parameters, of the model. With recursive or other type of autoregressive models, the gradient calculation at time $t$ ends up depending on the gradient at $t-1, t-2, ...,$ and to $t=0$. This ends up being computationally expensive, but more importantly can lead to "vanishing" or "exploding" gradient problems, where many gradients are multiplied together and either blow up or shrink. See LINK_TO_RECURSIVE_GRADIENT_LATEX for more info...

RRNs and other timeseries neural network architectures* get around this issue by approximating the gradient in more stable ways. In addition to many model architecture and hyperparameter options, timeseries neural networks use two main ways of restructuring the input data.

* **Sequence Length:** The input data is divided into smaller collections of ordered events, known as sequences. When calculating the gradient with respect to the model weights, the gradient only looks back this number of timesteps. Also known as `timesteps`, `sequence_length`, or just "sample" in `tensorflow` functions and related literature. For a sequence length of 3, the unique sequences of length 3 are: `[1,2,3], [2,3,4], ..., [T-2, T-1, T]`, for a total number of sequences `T-timesteps+1`

* **Batch size:** Sequences are grouped together into batches. The batch size determines how many sequences the network processes in a single step of calculating loss and updating weights. Used as `batch_size` in `tensorflow`.

The total number of batches is therefore determined from the total number of observations $T$ and the batch size. In a single batch, the loss is typically calculated for each sequence and then averaged to produce a single value. Then, the gradient of the loss with respect to the parameters (weights and biases) is computed for each sequence in the batch. So each batch will have a single gradient calculation that is the sum of the gradients of each sequence in the batch.

**Note:* these same data principles apply to more complex versions of timeseries neural network layers, such as LSTM and GRU.

## Stateless vs Stateful Networks

RNNs have a hidden state that represents the recurrent layer output at a previous time. There is a weight and bias at each RNN cell that determines the relative contribution of the previous output to the current output. When updating weights in RNNs, there are two main types of training scheme:

**Stateless:** the hidden state is reset to the initial state (often zero) at the start of each new sequence in a batch. So, the network treats each sequence independently, and no information is carried over in time between sequences. These models are simpler, but work better when time dependence is relatively short.
* **Input Data Shape:** (`n_sequence`, `timesteps`, `features`), where `n_sequence` is total number of sequences (a function of total observed times `T` and the user choice of timesteps). The input data does NOT need to be structured with batch size in stateless RNNs.
* **Tensorflow RNN Args:** for a stateless RNN, use the `input_shape` parameter, with `input_shape`=(`timesteps`, `features`). Then, `batch_size` can be declared in the fitting stage with `model.fit(X_train, batch_size = __)`. 

**Stateful:** the hidden states are carried over from one sequence to the next within a batch. Longer time dependencies can be learned in this way.
* **Input Data Shape:** (`batch_size`, `timesteps`, `features`). In order for the hidden state to be passed between sequences, the input data must be formatted using the `batch_size` hyperparameter.
* **Tensorflow RNN Args:** for a stateful RNN, use the `batch_input_shape` parameter, with `batch_input_shape`=(`batch_size`, `timesteps`, `features`)

## Examples

### Data Description

Consider $T=100$ observations of a variable in time $y_t$, so $t=1, ..., 100$. A feature matrix with $1$ features has dimensions $100\times 1$, and must be restructured for use in RNNs.

In [ ]:
T=100 # total number of times obseved

data = np.arange(0, 100).reshape(-1, 1)
# data = np.array([[f"x{i}"] for i in range(100)])


# Generate random response vector, needed by staircase func
y = np.arange(1, 101).reshape(-1, 1)

print(f"Response Data Shape: {y.shape}")
print("First 10 rows")
print(y[0:10])

# Print head of data
print(f"Feature Data Shape: {data.shape}")
print("First 10 rows")
data[0:10,:]

The rows of the input data array represent all features at a single timepoint. The first digit represents the feature number, and the second digit represents time point. So value $13$ represents feature 1 at time 3. 

### Single Batch Example

With a stateless RNN, the input data is structured to be of shape (`n_sequence`, `timesteps`, `features`). The `batch_size` is not needed to structure the data for a stateless RNN.

When using functions that expect `batch_size` to structure the data, an option is to set `batch_size` to be some large number greater than the total number of observed times $T$, so that all the data is guarenteed to be in one batch. *NOTE:* here we trick the function by using a large batch size, but `batch_size` could still be declared at the fitting stage of the model.

Suppose in we use `timesteps=5`, so we would get sequences of times `[1,2,3,4,5]` for the first sequence, then `[2,3,4,5,6]` for the next, and so on until `[96,97,98,99,100]`. 

Thus, there are `100-5+1=96` possible ordered sequences of length `5`. 

We need to structure the input data to the RNN to be of shape (96, 5, 3). *Note:* since the model is stateless, and the sequences are treated independently, the actual order of the sequences doesn't matter.

For a stateless RNN, the batches could consist of any collection of the sequences, since the sequences are indepenent. 

We want all of the data sequences to be in a single batch, but number of batches is not a direct user input for most built-in functions. To get around this, we make the batch size some number larger than the total number of observed times.

We now recreate this using the custom `staircase` function, which produces the same results for the input data:

In [ ]:
X_train, y_train = staircase(data, y, timesteps = 5, datapoints = len(y), verbose=True)

In [ ]:
print(X_train.shape)
print(y_train.shape)

The first input sequence will be 5 observations of the features starting at time 0:

In [ ]:
print("Sequence 1:")
print(X_train[0,:,:])

The second input sequence will be 5 observations of the features starting at time 1:

In [ ]:
print("Sequence 2:")
X_train[1,:,:]

In this implementation, we structure the input data as all possible sequences of length 5, but there is no requirement to do it this way. With a stateless RNN, you can use any combination of sequences. The input data does not need to be highly structured, you can put any combination of sequences you want in an array.

### Stateful Multi-Batch Example

We now need the data in the format (`batch_size`, `timesteps`, `features`) for each batch. A stateful RNN maintains a hidden state between batches. So sequence $i$ in batch $j$ needs to be a continuation of sequence $i$ in batch $j-1$.

In [ ]:
X_train, y_train = staircase_2(data, y, timesteps = 5, batch_size = 32, verbose=False)

The first input sequence will again be 5 observations of the features starting at time 0:

In [ ]:
print("Sequence 1, Batch 1:")
X_train[0,:,:]

In [ ]:
print("Sequence 2, Batch 1:")
X_train[1,:,:]

Since the batch size is 32, the 33rd input sequence will be a continuation of the 1st input sequence. So the 33rd input sequence starts at time 5:

In [ ]:
print("Sequence 1, Batch 2:")
X_train[0+32,:,:]

In [ ]:
print("Sequence 2, Batch 2:")
X_train[1+32,:,:]

In [ ]:
print("Sequence 1, Batch 3:")
X_train[32+32,:,:]

In [ ]:
print("Sequence 2, Batch 3:")
X_train[33+32,:,:]

By setting a RNN to be stateful with batch size 32, in the first batch the model will run each sequence through the model. The hidden state at the end of sequence $i$ is used as the initial hidden state for seqeunce $i$ in the next batch.

In this example we again structured the data to use all possible sequences of length 5. So within batch 1, the 1st sequence starts at time zero, the next sequence starts at time 1. But within a batch, there does not need to be any temporal relationship between the sequences. The only requirement is that the $i^{th}$ sequence within each batch lines up.

## Multiple Timeseries Data Structure

Before we showed how to structure the input data if we have observations from 1 timeseries. In spatial contexts, there will be observation at multiple locations. There are many different ways to implement training for RNNs using multiple timeseries.

### Example: Stateful & Locations Equals Batch Size, Same Start Time

As a simplifying first example, suppose we set the batch size equal to the number of unique locations. So suppose we have 3 timeseries of observations at 3 unique locations. In this example, we will suppose they occur at the same time, but the data structure used here does not depend on that. The only temporal dependence is between sequences that will share a hidden state across batches in a stateful RNN. We include a second column in the feature matrix for location index below, values 1, 2, or 3. 

In [ ]:
data2 = np.column_stack(
    (np.concatenate((np.arange(0, 100), np.arange(0, 100), np.arange(0, 100))),
    np.concatenate((np.repeat(1, 100), np.repeat(2, 100), np.repeat(3, 100))))
)

In [ ]:
print('First 10 observations at location 1:')
data2[0:10,:]

In [ ]:
print('First 10 observations at location 2:')
data2[100:110,:]

In [ ]:
print('First 10 observations at location 3:')
data2[200:210,:]

In this example, we construct a dataset with `batch_size` = 3.

In [ ]:
X1, y1 = staircase_2(data2[data2[:,1] == 1], y, timesteps = 5, batch_size = 1, verbose=False)
X2, y2 = staircase_2(data2[data2[:,1] == 2], y, timesteps = 5, batch_size = 1, verbose=False)
X3, y3 = staircase_2(data2[data2[:,1] == 3], y, timesteps = 5, batch_size = 1, verbose=False)

Xs = [X1, X2, X3]

In [ ]:
[Xi.shape[0] for Xi in Xs]

In [ ]:
locs = len(Xs)
XX = np.empty((Xs[0].shape[0]*locs, 5, 2))

In [ ]:
for i in range(0,locs):
    XX[i::locs] =  Xs[i]

In [ ]:
print("Sequence 1, Batch 1")
print(XX[0,:,:])
print("Sequence 1, Batch 2")
print(XX[3,:,:])

In [ ]:
print("Sequence 2, Batch 1")
print(XX[1,:,:])
print("Sequence 2, Batch 2")
print(XX[4,:,:])

In [ ]:
print("Sequence 3, Batch 1")
print(XX[2,:,:])
print("Sequence 3, Batch 2")
print(XX[5,:,:])

### Example: Stateful & Locations Equals Batch Size, Staggered Start Time

In the previous example, within a batch the sequences all start at the same time. This can lead to over-reliance on the particular ordering of the data. In this next example, we will use the same data from 3 locations as before, but we will stagger the start time of the sequences. This will result in losing a sequence at the end of the timeseries that are offset, so we filter out the data to match in dimensions.

In [ ]:
X1, y1 = staircase_2(data2[(data2[:,1] == 1) & (data2[:,0]>= 0)], y, timesteps = 5, batch_size = 1, verbose=False)
X2, y2 = staircase_2(data2[(data2[:,1] == 2) & (data2[:,0]>= 1)], y, timesteps = 5, batch_size = 1, verbose=False)
X3, y3 = staircase_2(data2[(data2[:,1] == 3) & (data2[:,0]>= 2)], y, timesteps = 5, batch_size = 1, verbose=False)

Xs = [X1, X2, X3]

In [ ]:
lens = [Xi.shape[0] for Xi in Xs]
print(lens)
print(min(lens))

In [ ]:
# Filter each array to be same length
min_shape = min(lens)
Xs = [Xi[:min_shape] for Xi in Xs]

In [ ]:
[Xi.shape[0] for Xi in Xs]

In [ ]:
locs = len(Xs)
XX = np.empty((Xs[0].shape[0]*locs, 5, 2))

for i in range(0,locs):
    XX[i::locs] =  Xs[i]

In [ ]:
print("Sequence 1, Batch 1")
print(XX[0,:,:])
print("Sequence 1, Batch 2")
print(XX[3,:,:])

In [ ]:
print("Sequence 2, Batch 1")
print(XX[1,:,:])
print("Sequence 2, Batch 2")
print(XX[4,:,:])

In [ ]:
print("Sequence 3, Batch 1")
print(XX[2,:,:])
print("Sequence 3, Batch 2")
print(XX[5,:,:])

### Example: Fewer Locations than Batch Size

In [ ]:
# X1, y1 = staircase_2(data2[(data2[:,1] == 1) & (data2[:,0]>= 0)], y, timesteps = 5, batch_size = 1, verbose=False)
# X2, y2 = staircase_2(data2[(data2[:,1] == 2) & (data2[:,0]>= 1)], y, timesteps = 5, batch_size = 1, verbose=False)
# X3, y3 = staircase_2(data2[(data2[:,1] == 3) & (data2[:,0]>= 2)], y, timesteps = 5, batch_size = 1, verbose=False)

# Xs = [X1, X2, X3]

### Example: More Locations than Batch Size

In [ ]:
data_config = {
    'locs': 3,
    'start_times': [0,1,2], # relative to first observation, must match number of locs
    'total_hours': 100, # total number of hours to use from data
    'batch_size': 2
}

In [ ]:
locs = len(np.unique(data2[:,1]))
print(f"Unique Locations: {locs}")

In [ ]:
batch_size = 2

In [ ]:
j = 0 # starting location index
for j in range():
    X1, y1 = staircase_2(data2[(data2[:,1] == 1) & (data2[:,0]>= 0)], y, timesteps = 5, batch_size = 1, verbose=False)
    X2, y2 = staircase_2(data2[(data2[:,1] == 2) & (data2[:,0]>= 1)], y, timesteps = 5, batch_size = 1, verbose=False)
    X3, y3 = staircase_2(data2[(data2[:,1] == 3) & (data2[:,0]>= 2)], y, timesteps = 5, batch_size = 1, verbose=False)
    
    Xs = [X1, X2, X3]

## References

https://d2l.ai/chapter_recurrent-neural-networks/bptt.html

https://www.tensorflow.org/guide/keras/working_with_rnns#cross-batch_statefulness

Tensorflow `timeseries_dataset_from_array` tutorial: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array

Wiki BPTT: https://en.wikipedia.org/wiki/Backpropagation_through_time#:~:text=Backpropagation%20through%20time%20(BPTT)%20is,independently%20derived%20by%20numerous%20researchers.

https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/